In [45]:
import pandas as pd
import os

In [46]:
directory = 'plays'
plays_agg = []
for i in range(29):
    file_name = f'plays_{i}.csv'
    path = os.path.join(directory, file_name)
    try:
        plays = pd.read_csv(path)
    except FileNotFoundError:
        print(f'file {i} skipped')
        continue
    plays_agg.append(plays)

file 4 skipped
file 18 skipped
file 19 skipped
file 20 skipped


In [47]:
comb_plays = pd.concat(plays_agg)

In [48]:
comb_plays[comb_plays['actionType'] == 'Foul']['subType'].unique()

array(['Shooting', 'Personal', 'Offensive Charge', 'Defense 3 Second',
       'Loose Ball', 'Offensive', 'Personal Take', 'Double Personal',
       'Transition Take', 'Technical', 'Flagrant Type 2',
       'Double Technical', 'Away From Play', 'Flagrant Type 1'],
      dtype=object)